In [114]:
import os

import numpy as np
import pandas as pd
from tqdm import tqdm
import mlflow
from dotenv import load_dotenv

load_dotenv("../../.env")

True

In [115]:
def find_merchant_folders(input_folder):
    return [os.path.join(input_folder, d) for d in os.listdir(input_folder)
            if os.path.isdir(os.path.join(input_folder, d)) and d.startswith("merchant")]

In [116]:
def get_orders_by_merchant(folder_path):
    orders_folder = os.path.join(folder_path, "orders")

    usecols = ["transaction_id", "created_at", "merchant_id"]
    dtypes = {"transaction_id": "object", "merchant_id": "object"}
    orders_df = pd.concat([pd.read_csv(os.path.join(orders_folder, f),
                                       usecols=usecols,
                                       parse_dates=["created_at"],
                                       dtype=dtypes) for f in tqdm(os.listdir(orders_folder)) if f.endswith(".csv")],
                           ignore_index=True) if os.path.exists(orders_folder) else pd.DataFrame()
    return orders_df

In [117]:
def get_orders_df(input_folder):
    merchant_folders = find_merchant_folders(input_folder)
    dataframes = [get_orders_by_merchant(folder) for folder in merchant_folders]

    # Stack all merchant DataFrames together
    final_df = pd.concat(dataframes, ignore_index=True) if dataframes else pd.DataFrame()
    return final_df

In [118]:
def get_chargebacks_df(input_folder):
    merchant_folders = find_merchant_folders(input_folder)

    usecols = ["transaction_id", "created_at"]
    dtypes = {"transaction_id": "object"}

    dfs = []
    for folder in merchant_folders:
        merchant_id = os.path.basename(folder).split("_")[1]
        merchant_chargebacks = pd.read_csv(os.path.join(folder, "chargebacks.csv"),
                               usecols=usecols,
                               parse_dates=["created_at"],
                               dtype=dtypes)
        merchant_chargebacks["merchant_id"] = merchant_id
        dfs.append(merchant_chargebacks)
    return pd.concat(dfs, ignore_index=True)

def get_fraud_alerts_df(input_folder):
    merchant_folders = find_merchant_folders(input_folder)

    usecols = ["transaction_id", "created_at"]
    dtypes = {"transaction_id": "object"}

    dfs = []
    for folder in merchant_folders:
        merchant_id = os.path.basename(folder).split("_")[1]
        merchant_fraud_alerts = pd.read_csv(os.path.join(folder, "fraud_alerts.csv"),
                               usecols=usecols,
                               parse_dates=["created_at"],
                               dtype=dtypes)
        merchant_fraud_alerts["merchant_id"] = merchant_id
        dfs.append(merchant_fraud_alerts)
    return pd.concat(dfs, ignore_index=True)

In [119]:
def aggregate_and_merge(orders_df: pd.DataFrame,
                        chargebacks_df: pd.DataFrame,
                        fraud_alerts_df: pd.DataFrame) -> pd.DataFrame:
    orders_df["created_at"] = pd.to_datetime(orders_df["created_at"]).dt.floor("D")
    chargebacks_df["created_at"] = pd.to_datetime(chargebacks_df["created_at"]).dt.floor("D")
    fraud_alerts_df["created_at"] = pd.to_datetime(fraud_alerts_df["created_at"]).dt.floor("D")

    orders_agg = orders_df.groupby(["merchant_id", "created_at"]).agg(orders_count=("transaction_id", "count")).reset_index()
    orders_agg.rename(columns={"created_at": "date"}, inplace=True)

    chargebacks_agg = chargebacks_df.groupby(["merchant_id", "created_at"]).agg(chargebacks_count=("transaction_id", "count")).reset_index()
    chargebacks_agg.rename(columns={"created_at": "date"}, inplace=True)

    fraud_alerts_agg = fraud_alerts_df.groupby(["merchant_id", "created_at"]).agg(fraud_alerts_count=("transaction_id", "count")).reset_index()
    fraud_alerts_agg.rename(columns={"created_at": "date"}, inplace=True)

    df = pd.merge(orders_agg, chargebacks_agg, on=["merchant_id", "date"], how="outer")
    df = pd.merge(df, fraud_alerts_agg, on=["merchant_id", "date"], how="outer")
    df[["orders_count", "chargebacks_count", "fraud_alerts_count"]] = df[["orders_count", "chargebacks_count", "fraud_alerts_count"]].fillna(0)
    df["date"] = pd.to_datetime(df["date"])

    return df

In [120]:
def compute_rolling_risk(group, window=30):
    group = group.sort_values("date")
    group["orders_roll_sum_30"] = group["orders_count"].rolling(window=window, min_periods=1).sum().shift(1)
    group["risk_events_roll_sum_30"] = (group["chargebacks_count"] + group["fraud_alerts_count"]).rolling(window=window, min_periods=1).sum().shift(1)
    group["rolling_risk_metric_30"] = 100 * (group["risk_events_roll_sum_30"] / (group["orders_roll_sum_30"] + 1e-6))
    return group


def create_extended_lag_features(group, lags=[1, 3, 5, 7, 14, 30, 60, 90]):
    group = group.sort_values("date")
    # Create lags for orders, chargebacks, fraud alerts and their sum (risk events)
    for lag in lags:
        group[f'orders_lag_{lag}'] = group['orders_count'].shift(lag)
        group[f'chargebacks_lag_{lag}'] = group['chargebacks_count'].shift(lag)
        group[f'fraud_alerts_lag_{lag}'] = group['fraud_alerts_count'].shift(lag)
        group[f'risk_events_lag_{lag}'] = (group["chargebacks_count"] + group["fraud_alerts_count"]).shift(lag)
    return group


def create_multiple_window_features(group, windows=[7, 14, 30, 60, 90]):
    group = group.sort_values("date")
    for w in windows:
        group[f'orders_roll_sum_{w}'] = group['orders_count'].rolling(window=w, min_periods=1).sum().shift(1)
        group[f'risk_events_roll_sum_{w}'] = (group['chargebacks_count'] + group['fraud_alerts_count']).rolling(window=w, min_periods=1).sum().shift(1)
    return group


def create_exponential_decay_features(group, span=7):
    group = group.sort_values("date")
    group["orders_ewm_mean"] = group["orders_count"].ewm(span=span, adjust=False).mean().shift(1)
    group["chargebacks_ewm_mean"] = group["chargebacks_count"].ewm(span=span, adjust=False).mean().shift(1)
    group["fraud_alerts_ewm_mean"] = group["fraud_alerts_count"].ewm(span=span, adjust=False).mean().shift(1)
    group["risk_events_ewm_mean"] = (group["chargebacks_count"] + group["fraud_alerts_count"]).ewm(span=span, adjust=False).mean().shift(1)
    group["rolling_risk_metric_ewm"] = 100 * (group["risk_events_ewm_mean"] / (group["orders_ewm_mean"] + 1e-6))
    return group


def create_ratio_features(group, windows=[7, 14, 30, 60, 90]):
    group = group.sort_values("date")
    for w in windows:
        group[f'risk_to_orders_ratio_{w}'] = group[f'risk_events_roll_sum_{w}'] / (group[f'orders_roll_sum_{w}'] + 1e-6)
    if "risk_to_orders_ratio_14" in group.columns and "risk_to_orders_ratio_7" in group.columns:
        group['risk_ratio_diff_14_7'] = group['risk_to_orders_ratio_14'] - group['risk_to_orders_ratio_7']
    if "risk_to_orders_ratio_30" in group.columns and "risk_to_orders_ratio_14" in group.columns:
        group['risk_ratio_diff_30_14'] = group['risk_to_orders_ratio_30'] - group['risk_to_orders_ratio_14']
    return group


def create_time_based_and_seasonal_features(df: pd.DataFrame):
    df["day_of_week"] = df["date"].dt.dayofweek
    df["month"] = df["date"].dt.month
    df["date_ordinal"] = df["date"].map(pd.Timestamp.toordinal)
    df["day_of_year"] = df["date"].dt.dayofyear
    df["sin_day"] = np.sin(2 * np.pi * df["day_of_year"] / 365)
    df["cos_day"] = np.cos(2 * np.pi * df["day_of_year"] / 365)
    df["sin_month"] = np.sin(2 * np.pi * df["month"] / 12)
    df["cos_month"] = np.cos(2 * np.pi * df["month"] / 12)

    return df


In [121]:
def create_features(df: pd.DataFrame) -> pd.DataFrame:
    transforms = [compute_rolling_risk,
                  create_extended_lag_features,
                  create_multiple_window_features,
                  create_exponential_decay_features,
                  create_ratio_features]
    for transform in transforms:
        df = df.groupby("merchant_id", group_keys=False).apply(transform)

    df = create_time_based_and_seasonal_features(df)

    for lag in [1, 2, 3]:
        df[f'rolling_risk_metric_lag_{lag}'] = df.groupby("merchant_id")["rolling_risk_metric_30"].shift(lag)

    df.dropna(inplace=True)
    df.reset_index(drop=True, inplace=True)

    return df

In [122]:
import boto3
import io

def save_dataframe_to_s3(df: pd.DataFrame, bucket_name: str, file_key: str):
    buffer = io.BytesIO()
    df.to_parquet(buffer, engine='pyarrow', index=False)
    buffer.seek(0)

    s3_client = boto3.client(
        's3',
        aws_access_key_id=os.getenv("AWS_ACCESS_KEY_ID"),
        aws_secret_access_key=os.getenv("AWS_SECRET_ACCESS_KEY")
    )

    s3_client.upload_fileobj(buffer, bucket_name, file_key)
    print(f"File saved to s3://{bucket_name}/{file_key}")

In [123]:
def data_processing_pipeline(data_folder: str) -> pd.DataFrame:
    print("Prepare orders df")
    orders_df = get_orders_df(data_folder)
    print("Prepare chargebacks df")
    chargebacks_df = get_chargebacks_df(data_folder)
    print("Prepare fraud alerts df")
    fraud_alerts_df = get_fraud_alerts_df(data_folder)

    print("Aggregate and merge")
    df = aggregate_and_merge(orders_df, chargebacks_df, fraud_alerts_df)
    print("Create new features")
    df = create_features(df)
    print("Saving to S3")
    save_dataframe_to_s3(df, os.getenv("BUCKET_NAME"), "data/processed_data.parquet")

    return df

In [124]:
processed_df = data_processing_pipeline("../data")

Prepare orders df


100%|██████████| 22/22 [01:59<00:00,  5.44s/it]


Prepare chargebacks df
Prepare fraud alerts df
Aggregate and merge
Create new features
Saving to S3


C:\Users\andrk\AppData\Local\Temp\ipykernel_12544\1641976080.py:8: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby("merchant_id", group_keys=False).apply(transform)
C:\Users\andrk\AppData\Local\Temp\ipykernel_12544\1641976080.py:8: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby("merchant_id", group_keys=False).apply(transform)
C:\Users\andrk\AppData\Local\Temp\ipykernel_12544\16

File saved to s3://int20h-data/data/processed_data.parquet


In [130]:
from xgboost import XGBRegressor
import optuna
from sklearn.metrics import r2_score
from sklearn.model_selection import TimeSeriesSplit


def read_dataframe_from_s3(bucket_name: str, file_key: str) -> pd.DataFrame:
    s3_client = boto3.client(
            's3',
            aws_access_key_id=os.getenv("AWS_ACCESS_KEY_ID"),
            aws_secret_access_key=os.getenv("AWS_SECRET_ACCESS_KEY")
        )

    # Download file into a buffer
    buffer = io.BytesIO()
    s3_client.download_fileobj(bucket_name, file_key, buffer)
    buffer.seek(0)  # Reset buffer position

    # Read Parquet file into DataFrame
    df = pd.read_parquet(buffer, engine='pyarrow')
    return df


def get_training_data(df: pd.DataFrame, feature_cols: list[str], target_col: str):
    return df[feature_cols], df[target_col]


def select_hyperparameters(data, target):
    tscv = TimeSeriesSplit(n_splits=3)

    def objective(trial):
        learning_rate = trial.suggest_float("learning_rate", 0.01, 0.5)
        max_depth = trial.suggest_int("max_depth", 3, 15)
        n_estimators = trial.suggest_int("n_estimators", 100, 800, step=100)
        subsample = trial.suggest_float("subsample", 0.6, 1.0)
        colsample_bytree = trial.suggest_float("colsample_bytree", 0.6, 1.0)
        reg_alpha = trial.suggest_float("reg_alpha", 0.0, 10.0)
        reg_lambda = trial.suggest_float("reg_lambda", 1.0, 100.0)

        model = XGBRegressor(
            random_state=42,
            learning_rate=learning_rate,
            max_depth=max_depth,
            n_estimators=n_estimators,
            subsample=subsample,
            colsample_bytree=colsample_bytree,
            reg_alpha=reg_alpha,
            reg_lambda=reg_lambda,
            n_jobs=-1
        )

        fold_scores = []
        for train_idx, val_idx in tscv.split(data):
            X_train, X_val = data.iloc[train_idx], data.iloc[val_idx]
            y_train, y_val = target.iloc[train_idx], target.iloc[val_idx]
            model.fit(X_train, y_train)
            preds = model.predict(X_val)
            score = r2_score(y_val, preds)
            fold_scores.append(score)
        return np.mean(fold_scores)
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=30)

    best_params = study.best_trial.params
    print("Best hyperparameters:", best_params)
    print("Best CV R²:", study.best_trial.value)
    return best_params

def train_model(data: pd.DataFrame, target: pd.Series, best_params: dict):
    mlflow.set_tracking_uri(os.getenv("MLFLOW_TRACKING_URI_DEV"))
    mlflow.set_experiment("int20h-xdboobs")
    with mlflow.start_run(run_name=os.getenv("MLFLOW_RUN_NAME")):
        final_model = XGBRegressor(random_state=42, **best_params)
        final_model.fit(data, target)

        train_r2 = r2_score(target, final_model.predict(data))
        mlflow.log_metric("R2", train_r2)
        for key, value in best_params.items():
            mlflow.log_param(key, value)
        mlflow.xgboost.log_model(final_model, "xgboost_model")


In [135]:
def get_feature_columns():
    feature_columns = [
    "rolling_risk_metric_lag_1", "rolling_risk_metric_lag_2", "rolling_risk_metric_lag_3",
    # Extended lag features
    "orders_lag_1", "orders_lag_3", "orders_lag_5", "orders_lag_7", "orders_lag_14", "orders_lag_30",
    "chargebacks_lag_1", "chargebacks_lag_3", "chargebacks_lag_5", "chargebacks_lag_7", "chargebacks_lag_14", "chargebacks_lag_30",
    "fraud_alerts_lag_1", "fraud_alerts_lag_3", "fraud_alerts_lag_5", "fraud_alerts_lag_7", "fraud_alerts_lag_14", "fraud_alerts_lag_30",
    "risk_events_lag_1", "risk_events_lag_3", "risk_events_lag_5", "risk_events_lag_7", "risk_events_lag_14", "risk_events_lag_30",
    # Rolling sums over multiple windows
    "orders_roll_sum_7", "orders_roll_sum_14", "orders_roll_sum_30", "orders_roll_sum_60", "orders_roll_sum_90",
    "risk_events_roll_sum_7", "risk_events_roll_sum_14", "risk_events_roll_sum_30", "risk_events_roll_sum_60", "risk_events_roll_sum_90",
    # Ratio features
    "risk_to_orders_ratio_7", "risk_to_orders_ratio_14", "risk_to_orders_ratio_30", "risk_to_orders_ratio_60", "risk_to_orders_ratio_90",
    "risk_ratio_diff_14_7", "risk_ratio_diff_30_14",
    # Exponential decay features
    "orders_ewm_mean", "chargebacks_ewm_mean", "fraud_alerts_ewm_mean", "risk_events_ewm_mean", "rolling_risk_metric_ewm",
    # Time and seasonal features
    "day_of_week", "month", "date_ordinal",
    "sin_day", "cos_day", "sin_month", "cos_month"
    ]
    return feature_columns

In [131]:
def model_training_pipeline():
    processed_df = read_dataframe_from_s3(os.getenv("BUCKET_NAME"), "data/processed_data.parquet")

    feature_columns = get_feature_columns()
    target_column = "rolling_risk_metric_30"

    data, target = get_training_data(processed_df, feature_columns, target_column)
    best_params = select_hyperparameters(data, target)
    train_model(data, target, best_params)

model_training_pipeline()

[I 2025-03-02 01:05:00,656] A new study created in memory with name: no-name-b7f3a42b-fbc5-460c-b545-54abd804dea3
[I 2025-03-02 01:05:01,196] Trial 0 finished with value: -0.0950136136628154 and parameters: {'learning_rate': 0.4472595491978774, 'max_depth': 15, 'n_estimators': 200, 'subsample': 0.6768120592977138, 'colsample_bytree': 0.9985071793993816, 'reg_alpha': 8.758320332067736, 'reg_lambda': 86.13472366830968}. Best is trial 0 with value: -0.0950136136628154.
[I 2025-03-02 01:05:02,044] Trial 1 finished with value: 0.05574951804016196 and parameters: {'learning_rate': 0.31539615441631386, 'max_depth': 11, 'n_estimators': 400, 'subsample': 0.8181115150693915, 'colsample_bytree': 0.7147940350936048, 'reg_alpha': 2.1668681456004446, 'reg_lambda': 72.33422607151724}. Best is trial 1 with value: 0.05574951804016196.
[I 2025-03-02 01:05:03,006] Trial 2 finished with value: -0.023358357933212726 and parameters: {'learning_rate': 0.420326968354153, 'max_depth': 10, 'n_estimators': 600, 

Best hyperparameters: {'learning_rate': 0.10909373319085952, 'max_depth': 3, 'n_estimators': 800, 'subsample': 0.7494334255183932, 'colsample_bytree': 0.7309266965702423, 'reg_alpha': 0.004675858754265516, 'reg_lambda': 46.766727365670086}
Best CV R²: 0.18825719440992336


C:\Users\andrk\PycharmProjects\int20h-2025-xdboobs\pipelines\.venv\Lib\site-packages\xgboost\core.py:158: UserWarning: [01:05:44] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\c_api\c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  warnings.warn(smsg, UserWarning)
2025/03/02 01:06:08 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
2025/03/02 01:06:08 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run languid-hound-649 at: https://b2fd-91-123-155-204.ngrok-free.app/#/experiments/531295962442669067/runs/b1c7997fd0bc4a8580f5eb30fffe2027
🧪 View experiment at: https://b2fd-91-123-155-204.ngrok-free.app/#/experiments/531295962442669067


In [138]:
def create_test_df(submission_df: pd.DataFrame, train_df: pd.DataFrame):
    submission_df["merchant_id"] = submission_df["merchant_id_day"].apply(lambda x: x.split("_")[0])
    submission_df["date"] = pd.to_datetime(submission_df["merchant_id_day"].apply(lambda x: x.split("_")[1]))

    feature_columns = get_feature_columns()

    test_rows = []
    for idx, row in submission_df.iterrows():
        merchant_id = row["merchant_id"]
        forecast_date = row["date"]

        merchant_hist = train_df[train_df["merchant_id"] == merchant_id]
        if merchant_hist.empty:
            baseline = {feat: 0 for feat in feature_columns}
        else:
            baseline = merchant_hist.sort_values("date").iloc[-1].to_dict()

        baseline["date_ordinal"] = forecast_date.toordinal()
        baseline["day_of_week"] = forecast_date.dayofweek
        baseline["month"] = forecast_date.month

        test_rows.append(baseline)

    return pd.DataFrame(test_rows)[feature_columns]


def create_submission(submission_df: pd.DataFrame, model_uri: str):
    processed_df = read_dataframe_from_s3(os.getenv("BUCKET_NAME"), "data/processed_data.parquet")

    test_df = create_test_df(submission_df, processed_df)
    model = mlflow.pyfunc.load_model(model_uri)

    test_df["risk_metric_pred"] = model.predict(test_df)

    final_submission_df = pd.DataFrame({
        "merchant_id_day": submission_df["merchant_id_day"],
        "risk_metric": test_df["risk_metric_pred"]
    })
    return final_submission_df

In [140]:
final_submission_df = create_submission(pd.read_csv("../data/sample_submission.csv"),
                  "s3://int20h-data/mlflow_artifacts/531295962442669067/b1c7997fd0bc4a8580f5eb30fffe2027/artifacts/xgboost_model")

In [143]:
final_submission_df.to_csv("submission.csv", index=False)

In [144]:
final_submission_df

,merchant_id_day,risk_metric
0,8508273933093481573_2024-08-12,2.883748
1,8508273933093481573_2024-08-13,2.883748
2,8508273933093481573_2024-08-14,2.885600
3,8508273933093481573_2024-08-15,2.885795
4,8508273933093481573_2024-08-16,2.887431
...,...,...
541,11019390644840797009_2025-02-05,0.894275
542,11019390644840797009_2025-02-06,0.894295
543,11019390644840797009_2025-02-07,0.895400
544,11019390644840797009_2025-02-08,0.895146
